In [94]:
import numpy as np
import pandas as pd

In [95]:
df = pd.read_csv("/Users/riyaazkhan/Documents/Imperial_Maths/Algorithmic_Trading_Club/Projects/Horse-Racing-Project/Data/Processed/full_feature_engineered_data.csv")

/var/folders/zh/vm69w9xx2dz7lj6rxj4mc08r0000gn/T/ipykernel_81804/4218471939.py:1: DtypeWarning: Columns (9,27,37,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/riyaazkhan/Documents/Imperial_Maths/Algorithmic_Trading_Club/Projects/Horse-Racing-Project/Data/Processed/full_feature_engineered_data.csv")


In [179]:
(df.columns)

Index(['age', 'saddle', 'isFav', 'overWeight', 'outHandicap', 'RPR', 'TR',
       'OR', 'weight', 'res_win',
       ...
       'jump_flat', 'jump_hurdles', 'jump_chase', 'age_juvenile',
       'age_open_all', 'age_mixed', 'age_open_young', 'age_young_only',
       'age_veteran', 'age_unknown'],
      dtype='object', length=130)

In [97]:
features = [
    'horse_course_winrate', 
    'horse_jumptype_win_Rate', 
    'horse_rclass_win_Rate', 
    'horse_distanceband_win_Rate', 
    'horse_class_win_Rate',
    'horse_coursesurface_win_Rate', 
    'horse_saddlerankbin_win_Rate', 
    'horse_bandbin_win_Rate', 
    
    'jockey_jumptype_win_Rate',
    'jockey_course_win_Rate', 
    'horse_jockey_win_Rate',
    'jockey_distanceband_winrate', 
    
    'trainer_course_win_Rate', 
    'trainer_jumptype_win_Rate',
    'trainer_class_winrate', 
    'trainer_saddlerankbin_winrate',
    
    'father_course_winrate', 
    'father_jumptype_winrate', 
    'father_distanceband_winrate', 
    'father_coursesurface_winrate', 
    'father_class_winrate', 
    'father_band_winrate',
    
    'mother_course_winrate', 
    'mother_jumptype_winrate', 
    
    'gfather_course_winrate', 
    'gfather_jumptype_winrate', 
]

In [98]:
#save_path = "/Users/riyaazkhan/Documents/Imperial_Maths/Algorithmic_Trading_Club/Projects/Horse-Racing-Project/Data/Processed/"
#df['key'] = df['date'].astype(str) + '_' + df['horseName'].astype(str)

#for feat in features:
#    feat_df = pd.read_csv(f"{save_path}/{feat}.csv")
    
#    # Fix: drop or aggregate duplicates
#    if feat_df['key'].duplicated().any():
#        print(f"WARNING: {feat} has duplicated keys. Deduplicating...")
#        feat_df = feat_df.groupby('key').mean(numeric_only=True).reset_index()
#
#    df = df.merge(feat_df, on='key', how='left')


In [99]:
outcome_df = pd.concat([df['res_win'],df['res_place']],axis=1)

In [100]:
import re
import numpy as np

# Make sure ages column is string and handle NaNs
ages_clean = df['ages'].fillna('').astype(str)

# Extract min and max ages using vectorized regex
df['race_min_age'] = ages_clean.str.extract(r'(\d+)', expand=False).astype(float)
df['race_max_age'] = ages_clean.str.extract(r'-(\d+)', expand=False).astype(float)

# Handle patterns like '3yo+' (no upper limit → leave max as NaN)
is_plus = ages_clean.str.contains(r'yo\+')

# Handle patterns like '4yo' (single age = min and max are equal)
is_single = ages_clean.str.match(r'^\d+yo$')
df.loc[is_single, 'race_max_age'] = df.loc[is_single, 'race_min_age']

In [101]:
def age_band_category(row):
    min_age, max_age = row['race_min_age'], row['race_max_age']

    if pd.isna(min_age):
        return 'unknown'

    if max_age is not np.nan:
        if max_age <= 3:
            return 'juvenile'      # 2–3yo only
        elif max_age <= 5:
            return 'young_only'   # e.g., 3-5yo
    if min_age >= 6:
        return 'veteran'          # 6yo+, 7yo+, etc.
    elif min_age <= 3 and np.isnan(max_age):
        return 'open_young'       # e.g., 3yo+
    elif np.isnan(max_age):
        return 'open_all'         # 4yo+, 5yo+, etc.
    
    return 'mixed'

df['race_age_band'] = df.apply(age_band_category, axis=1)


In [103]:
#Columns needed for regression 
unnecessary_columns = ['rid','horseName','decimalPrice','trainerName',
                 'jockeyName','position','dist','weightSt','weightLb',
                 'father','mother','gfather','runners',
                'weightSt','weightLb','headGear','father',
                'mother','gfather','course','band','ages',
                'hurdles','prizes','countryCode','source_year',
                'prev_raceId','band_clean','close_finish','year',
                'res_place','prev_headgear','course_clean','margin']
#Original Index   
#Key

df = df.drop(columns=unnecessary_columns,axis=1)


                 

In [104]:
cols_with_toomany_nans = [col for col in df.columns if df[col].isna().sum() > 1000]
df[[f"{col}_nan" for col in cols_with_toomany_nans]] = df[cols_with_toomany_nans].isna().astype(int)


In [105]:
df = df.fillna(-1)

In [106]:
rclassmapping = {
    'Class 1':1,
    'Class 2':2,
    'Class 3':3,
    'Class 4':4,
    'Class 5':5,
    'Class 6':6,
    'Class 7':7,}
    
df['rclass'] = df['rclass'].map(rclassmapping)

In [107]:
condition_mapping = {
    'Firm': 1,
    'Good To Firm': 2,
    'Good': 3,
    'Good To Yielding': 4,
    'Yielding': 5,
    'Yielding To Soft': 6,
    'Good To Soft': 7,
    'Soft': 8,
    'Soft To Heavy': 9,
    'Heavy': 10,
    'Very Soft': 11,
    'Holding': 12,
    'Hard': 13,
    'Fast': 14,
    'Slow': 15,
    'Standard To Fast': 16,
    'Standard': 17,
    'Standard To Slow': 18,
    'Sloppy': 19,
    'Muddy': 20,
    'Frozen': 21,
    'Abandoned': 22
}

df['condition'] = df['condition'].map(condition_mapping).fillna(-1)

In [108]:
band_bins_mapping = {
    '≤60':1,
    '61-75':2,
    '76-90':3,
    '91-105':4,
    '106-120':5,
    '121-140':6,
    '141-175':7}

df['band_bins'] = df['band_bins'].map(band_bins_mapping)

In [109]:
ncond_mapping = {
    'Hard':-1,
    '0':0,
    '1':1,
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10,
    '11':11,
    '12':12,
    '13':13,
    '14':14,
    '15':15,
    '16':16,
    '17':17,
    '18':18,
    '19':19,
    '20':20}
    
df['ncond'] = df['ncond'].map(ncond_mapping)

In [110]:
days_last_race_mapping = {
    '0-14d':1,
    '15-30d':2,
    '31-60d':3,
    '61-90d':4,
    '91-180d':5,
    '181-365d':6,
    '365+d':7 }

df['days_since_last_race_binned'] = df['days_since_last_race_binned'].map(days_last_race_mapping)



In [111]:
df['saddle_rank_bin'].unique()
saddle_rank_bin_mapping = {
    '(0.024, 0.143]':1,
    '(0.143, 0.25]':2,
    '(0.25, 0.333]':3,
    '(0.333, 0.444]':4,
    '(0.444, 0.55]':5,
    '(0.55, 0.647]':6,
    '(0.647, 0.75]':7,
    '(0.75, 0.846]':8,
    '(0.846, 0.944]':9,
    '(0.944, 1.0]':10}

df['saddle_rank_bin'] = df['saddle_rank_bin'].map(saddle_rank_bin_mapping)
  

In [112]:
from pandas.api.types import CategoricalDtype
import pandas as pd

def one_hot_safe(df, col, categories, prefix, drop_first=False, dtype='int8', copy=False):
    # optional copy; default False for speed
    if copy:
        df = df.copy()

    # lock category space on the original series (no extra temp DataFrame)
    ctype = CategoricalDtype(categories=categories, ordered=False)
    s = df[col].astype(ctype)

    # build dummies directly with target dtype
    dums = pd.get_dummies(s, prefix=prefix, dtype=dtype)

    # choose expected columns (handles drop_first without per-column drops)
    keep_categories = categories[1:] if drop_first else categories
    expected = [f"{prefix}_{c}" for c in keep_categories]

    # add missing cols & order in one vectorized step (no Python loop)
    dums = dums.reindex(columns=expected, fill_value=0)

    # concat is a bit faster than join for this use
    return pd.concat([df.drop(columns=[col]), dums], axis=1)


In [113]:
df = one_hot_safe(df,'course_surface',['AW', 'Turf', 'Dirt'],'surface')
df = one_hot_safe(df,'race_jump_type',['flat', 'hurdles', 'chase'],'jump')

df = one_hot_safe(df,'race_age_band',['juvenile', 'open_all', 'mixed', 'open_young', 'young_only',
       'veteran', 'unknown'],'age')

In [114]:
import pandas as pd
import numpy as np

def has_non_numeric(df, col):
    converted = pd.to_numeric(df[col], errors='coerce')
    
    non_numeric_found = (converted.isna() & df[col].notna()).any()
    
    return non_numeric_found
def columns_with_non_numeric(df):
    return [col for col in df.columns if has_non_numeric(df, col)]

print(columns_with_non_numeric(df))


['date', 'distance_banding', 'track_layout']


In [115]:
track_layout_mapping = {
    'sharp':-1,
    'neutral':0,
    'galloping':1}

df['track_layout'] = df['track_layout'].map(track_layout_mapping)

In [116]:
distance_banding_mapping = {
    'short':1,
    'medium':2,
    'long':3
}

df['distance_banding'] = df['distance_banding'].map(distance_banding_mapping)

In [117]:
continuous_cols = [
    'age',                      # years, but often treated as continuous in regression
    'saddle',                   # actual saddle number (weight cloth)
    'overWeight',                # extra weight carried
    'RPR', 'TR', 'OR',           # ratings                    
    'weight',                    # carried weight (numeric measurement)
    'distance',                  # race distance
    'prize',                     # prize money
    'ncond',                     # race condition numeric measure
    'racers_per_race',           # size of field (can be treated continuous)
    'days_since_last_race',      # days since last run
    'saddle_rank_norm',          # continuous version of saddle rank
    'band_upper',                # numeric upper value of band          
    'rpr_last', 'rpr_mean_last_3', 'rpr_best_last_5', 'rpr_mean_last_20', 'rpr_trend',
    'prev_max_rpr', 'prev_rpr_diff_from_top',
    'TR_last', 'TR_mean_last_3', 'TR_best_last_5', 'TR_mean_last_20', 'TR_trend',
    'diff_TR_RPR_last', 'diff_TR_RPR_last_3',
    'prev_max_TR', 'prev_TR_diff_from_top',
    'OR_last', 'OR_mean_last_3', 'OR_best_last_5', 'OR_mean_last_20', 'OR_trend',
    'diff_OR_RPR_last', 'diff_OR_RPR_last_3',
    'prev_max_OR', 'prev_OR_diff_from_top'
]

In [118]:
import sklearn
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[continuous_cols] = scaler.fit_transform(df[continuous_cols])

In [119]:
import pandas as pd
import numpy as np

def has_non_numeric(df, col):
    converted = pd.to_numeric(df[col], errors='coerce')
    
    non_numeric_found = (converted.isna() & df[col].notna()).any()
    
    return non_numeric_found
def columns_with_non_numeric(df):
    return [col for col in df.columns if has_non_numeric(df, col)]

print(columns_with_non_numeric(df))


['date']


In [120]:
df = df.fillna(0)

In [156]:
print("Min:", df['betting_prob_win'].min())
print("Max:", df['betting_prob_win'].max())
print(len(df[df['betting_prob_win'] <= 0]))
print(len(df[df['betting_prob_win'] > 1]))
print(len(df))

Min: 0.0009990009990009
Max: 0.99009900990099
0
0
1888322


In [157]:
def check_betting_prob_consistency(df, prob_col='betting_prob_win', tol=1e-6):
    """Check if betting_prob_win correctly inverts to decimal odds."""
    
    # Step 1: Convert prob → decimal odds
    decimal_odds = 1 / df[prob_col]
    
    # Step 2: Convert decimal odds back to probability
    recon_prob = 1 / decimal_odds
    
    # Step 3: Compare with original
    diff = (df[prob_col] - recon_prob).abs()
    max_diff = diff.max()
    
    print(f"Original prob range: {df[prob_col].min():.6f} → {df[prob_col].max():.6f}")
    print(f"Decimal odds range: {decimal_odds.min():.2f} → {decimal_odds.max():.2f}")
    print(f"Max reconstruction diff: {max_diff:.8f}")
    
    if max_diff > tol:
        print("⚠️ WARNING: Betting prob may not match 1/decimal odds exactly.")
    else:
        print("✅ Betting prob is consistent with decimal odds.")

# Example usage
check_betting_prob_consistency(df)

Original prob range: 0.000999 → 0.990099
Decimal odds range: 1.01 → 1001.00
Max reconstruction diff: 0.00000000
✅ Betting prob is consistent with decimal odds.


In [122]:
df.to_csv("/Users/riyaazkhan/Documents/Imperial_Maths/Algorithmic_Trading_Club/Projects/Horse-Racing-Project/Data/LinearRegressionModelData.csv")

In [131]:
import pandas as pd

def clean_dates(df, date_col='date', dayfirst=True):
    s = df[date_col].astype(str).str.strip()

    # 1) Null out common sentinels/placeholders
    bad_tokens = {'', 'nan', 'NaN', 'None', '-1', '0', '0000-00-00'}
    s = s.mask(s.isin(bad_tokens))

    # 2) First pass: general parser
    parsed = pd.to_datetime(s, errors='coerce', dayfirst=dayfirst, utc=False)

    # 3) Second pass: numeric YYYYMMDD like 20240317
    mask_num = parsed.isna() & s.str.fullmatch(r'\d{8}')
    if mask_num.any():
        parsed.loc[mask_num] = pd.to_datetime(s[mask_num], format='%Y%m%d', errors='coerce')

    # Keep only rows with valid dates
    keep = ~parsed.isna()
    cleaned = df.loc[keep].copy()
    cleaned[date_col] = parsed[keep]

    dropped = (~keep).sum()
    return cleaned, int(dropped)

In [146]:
df, dropped = clean_dates(df, date_col='date', dayfirst=True)
print(f"Dropped {dropped} rows with bad dates. Remaining: {len(df)}")

Dropped 0 rows with bad dates. Remaining: 1888322


In [176]:
import numpy as np
import pandas as pd
from datetime import date
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# === Strict leakage cleaner ===
def remove_market_leakage(df, betting_prob_col='betting_prob_win'):
    """
    Remove features that leak market information from the model training set.
    Keeps betting_prob_win ONLY for ROI calculation, not as a feature.
    """
    df = df.copy()

    # Columns to drop: directly derived from market odds
    leakage_cols = [
        'isFav',                # favourite flag from market
        'decimalPrice',         # odds (already in betting_prob_win)
        'margin',               # sum of implied probs (market overround)
        'fav_rank', 'fav_rank_norm', 'fav_rank_bin',  # favourite rank encoding
        'price_diff', 'price_rank',                   # price-based engineered features
    ]

    # Drop them if present
    drop_found = [c for c in leakage_cols if c in df.columns]
    if drop_found:
        print(f"[leakage cleaner] Dropping market-derived cols: {drop_found}")
        df = df.drop(columns=drop_found)

    if betting_prob_col not in df.columns:
        raise ValueError(f"{betting_prob_col} is missing from dataframe!")

    return df

# === Backtest function ===
def backtest_exclude_market_use_as_benchmark(
    df,
    target_col='res_win',
    date_col='date',
    race_id_col=None,
    prob_col='betting_prob_win',
    test_fraction=0.2,
    edge_rel=0.15,
    min_book_p=0.01, max_book_p=0.99,
    random_state=42,
    sgd_max_iter=30, sgd_tol=1e-4
):
    """
    Train on *all numeric, non-market* features only.
    Use bookmaker probability ONLY to decide/place bets and settle with 1/book_p odds.
    """
    # 1) Remove leakage columns
    d = remove_market_leakage(df, betting_prob_col=prob_col)

    # 2) Dates: keep only parsed & not in the future
    d[date_col] = pd.to_datetime(d[date_col], errors='coerce')
    d = d.dropna(subset=[date_col])
    d = d[d[date_col] <= pd.Timestamp(date.today())]

    # 3) Clean bookmaker probs
    book_p = pd.to_numeric(d[prob_col], errors='coerce')
    if book_p.max(skipna=True) and 1 < book_p.max() <= 100:
        book_p = book_p / 100.0
    book_p = book_p.replace([np.inf, -np.inf], np.nan).clip(min_book_p, max_book_p)
    d = d.assign(_book_p=book_p).dropna(subset=['_book_p', target_col])

    # 4) Chronological split
    d = d.sort_values(date_col).reset_index(drop=True)
    split_idx = int(len(d) * (1 - test_fraction))
    train_df = d.iloc[:split_idx].copy()
    test_df  = d.iloc[split_idx:].copy()

    # 5) Optional: forbid race overlap
    if race_id_col and race_id_col in d.columns:
        seen = set(train_df[race_id_col])
        test_df = test_df[~test_df[race_id_col].isin(seen)].copy()

    # 6) Feature selection: numeric only, excluding market + target/date/race_id
    exclude = {target_col, prob_col, '_book_p', date_col}
    if race_id_col: exclude.add(race_id_col)
    numeric_cols = d.select_dtypes(include=['number', 'bool']).columns.tolist()
    feature_cols = [c for c in numeric_cols if c not in exclude]
    if not feature_cols:
        raise ValueError("No numeric features left after excluding market + target/date/race_id.")

    # 7) Train/test matrices
    X_train = train_df[feature_cols].astype(np.float32)
    y_train = train_df[target_col].astype(int).to_numpy()
    X_test  = test_df[feature_cols].astype(np.float32)
    y_test  = test_df[target_col].astype(int).to_numpy()

    # Market (for betting only)
    book_p  = test_df['_book_p'].to_numpy(np.float64)
    odds    = 1.0 / book_p
    assert np.all((book_p > 0) & (book_p < 1)) and np.all(odds >= 1.0)

    # 8) Model training
    model = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
        SGDClassifier(loss='log_loss', penalty='l2',
                      max_iter=sgd_max_iter, tol=sgd_tol,
                      early_stopping=True, n_iter_no_change=3,
                      validation_fraction=0.1, random_state=random_state)
    )
    model.fit(X_train, y_train)
    model_p = np.clip(model.predict_proba(X_test)[:, 1], 0.0, 1.0)

    # 9) Bet rule
    bet_mask = model_p > (book_p * (1.0 + edge_rel))
    profit_each = np.where(bet_mask, np.where(y_test == 1, odds - 1.0, -1.0), 0.0)

    # 10) Metrics
    brier = mean_squared_error(y_test, model_p)
    n_test = len(y_test)
    n_bets = int(bet_mask.sum())
    total_profit = float(np.nansum(profit_each))
    roi = total_profit / n_bets if n_bets else 0.0

    real_win_rate_on_bets = y_test[bet_mask].mean() if n_bets else 0.0
    book_win_rate_on_bets = book_p[bet_mask].mean() if n_bets else 0.0
    rnd_wins = (np.random.default_rng(random_state).random(n_test) < book_p)
    rnd_profit = np.where(bet_mask, np.where(rnd_wins, odds - 1.0, -1.0), 0.0)
    rnd_roi = rnd_profit[bet_mask].mean() if n_bets else 0.0

    print(f"Train: {train_df[date_col].min().date()} → {train_df[date_col].max().date()}")
    print(f"Test : {test_df[date_col].min().date()} → {test_df[date_col].max().date()}  (rows={n_test})")
    print(f"Features used: {len(feature_cols)} (market excluded)")
    print(f"Brier={brier:.4f} | Bets={n_bets} ({n_bets/n_test*100:.2f}%) | Profit=£{total_profit:.2f} | ROI/bet=£{roi:.3f}")
    print(f"Win%% on bets (real vs book): {real_win_rate_on_bets:.3f} vs {book_win_rate_on_bets:.3f}")
    print(f"Random-by-book_p ROI/bet (sanity): {rnd_roi:.4f}")

    summary = {
        "brier": brier,
        "bets": n_bets,
        "bets_pct": (n_bets / n_test * 100.0) if n_test else 0.0,
        "total_profit": total_profit,
        "roi_per_bet": roi,
        "real_win_rate_on_bets": real_win_rate_on_bets,
        "book_win_rate_on_bets": book_win_rate_on_bets,
        "rnd_roi_by_book": rnd_roi,
        "n_features": len(feature_cols)
    }

    details = pd.DataFrame({
        "y_true": y_test,
        "model_prob": model_p,
        "book_prob": book_p,
        "decimal_odds": odds,
        "bet": bet_mask.astype(int),
        "profit": profit_each
    }, index=test_df.index)

    return model, summary, details


In [177]:
model, summary, details = backtest_exclude_market_use_as_benchmark(
    df,
    target_col='res_win',
    date_col='date',
    race_id_col='race_id',          # if you have it
    prob_col='betting_prob_win',
    test_fraction=0.2,
    edge_rel=0.15
)

[leakage cleaner] Dropping market-derived cols: ['isFav']
Train: 1997-01-01 → 2020-02-10
Test : 2020-02-10 → 2025-08-10  (rows=307215)
Features used: 126 (market excluded)
Brier=0.0661 | Bets=89919 (29.27%) | Profit=£62595.61 | ROI/bet=£0.696
Win%% on bets (real vs book): 0.177 vs 0.096
Random-by-book_p ROI/bet (sanity): 0.0044


In [ ]:
#Extra stuff

In [174]:
import pandas as pd
import numpy as np

def drop_market_leakage_strict(df, target_col, date_col, betting_prob_col, test_fraction=0.2, threshold=0.05):
    """
    Detect and drop features that leak market info by correlating with betting_prob_win.

    Args:
        df: DataFrame with all races
        target_col: str, target column name
        date_col: str, race date column
        betting_prob_col: str, market win probability column
        test_fraction: float, fraction for time-based test split
        threshold: float, absolute correlation threshold for dropping features
    Returns:
        cleaned_df, dropped_features
    """
    d = df.copy()
    d[date_col] = pd.to_datetime(d[date_col], errors='coerce')
    d = d.dropna(subset=[date_col, betting_prob_col, target_col])
    d = d.sort_values(date_col).reset_index(drop=True)

    # Split train/test by time
    split_idx = int(len(d) * (1 - test_fraction))
    test_df = d.iloc[split_idx:]

    # Only numeric columns for correlation check
    num_cols = [c for c in d.select_dtypes(include=[np.number]).columns
                if c not in [target_col, betting_prob_col]]

    # Correlation with market prob in test set
    corr_series = test_df[num_cols].corrwith(test_df[betting_prob_col])
    leak_feats = corr_series[abs(corr_series) > threshold].index.tolist()

    print(f"Detected {len(leak_feats)} possible leakage features (|corr| > {threshold}):")
    for f in leak_feats:
        print(f"  {f} (corr={corr_series[f]:.4f})")

    # Drop them
    cleaned_df = d.drop(columns=leak_feats)

    return cleaned_df, leak_feats


In [175]:
df_clean, dropped = drop_market_leakage_strict(
    df,
    target_col='res_win',
    date_col='date',
    betting_prob_col='betting_prob_win',
    test_fraction=0.2,
    threshold=0.05  # strict
)

print(f"\nRemaining features: {len(df_clean.columns)}")
print(f"Dropped features: {dropped}")



Detected 38 possible leakage features (|corr| > 0.05):
  saddle (corr=-0.2615)
  isFav (corr=0.7078)
  outHandicap (corr=-0.0734)
  RPR (corr=0.2663)
  TR (corr=0.1948)
  OR (corr=0.0828)
  weight (corr=0.0552)
  racers_per_race (corr=-0.2809)
  days_since_last_race (corr=-0.0520)
  saddle_rank (corr=-0.2657)
  saddle_rank_norm (corr=-0.1387)
  saddle_rank_bin (corr=-0.1513)
  trainer_win_encoded (corr=0.2733)
  jockey_win_encoded (corr=0.2824)
  horse_win_encoded (corr=0.2059)
  rpr_last (corr=0.1764)
  rpr_trend (corr=0.1112)
  prev_max_rpr (corr=0.0657)
  prev_rpr_diff_from_top (corr=0.0977)
  TR_last (corr=0.1407)
  TR_trend (corr=0.0598)
  diff_TR_RPR_last (corr=-0.0702)
  prev_max_TR (corr=0.0534)
  OR_last (corr=0.0860)
  OR_trend (corr=0.0640)
  prev_OR_diff_from_top (corr=0.0710)
  saddle_nan (corr=0.0586)
  outHandicap_nan (corr=0.0783)
  RPR_nan (corr=-0.0973)
  TR_nan (corr=-0.1078)
  saddle_rank_nan (corr=0.0586)
  saddle_rank_norm_nan (corr=0.0586)
  saddle_rank_bin_nan (